## How to Grid Search Hyperparameters for Deep Learning Models in Python with Keras

- https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

In [1]:
# Use scikit-learn to grid search the batch size and epochs
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from scikeras.wrappers import KerasRegressor

In [2]:
# Function to create model, required for KerasClassifier
def create_model(optimizer='adam'):
    
    # create model
    model = Sequential()
    model.add(Dense(12, input_shape=(8,), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    # return value
    return model

In [3]:
# fix random seed for reproducibility
tf.random.set_seed(seed=7)

In [4]:
# load dataset
dataset = np.loadtxt("dataset/diabetes.csv", delimiter=",")

In [5]:
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

In [6]:
# show to dataset
pd.concat([
    pd.DataFrame(X, columns=['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8']),
    pd.DataFrame(Y, columns=['Y'])
], axis=1)

,x1,x2,x3,x4,x5,x6,x7,x8,Y
0,6.0,148.0,72.0,35.0,0.0,33.6,0.627,50.0,1.0
1,1.0,85.0,66.0,29.0,0.0,26.6,0.351,31.0,0.0
2,8.0,183.0,64.0,0.0,0.0,23.3,0.672,32.0,1.0
3,1.0,89.0,66.0,23.0,94.0,28.1,0.167,21.0,0.0
4,0.0,137.0,40.0,35.0,168.0,43.1,2.288,33.0,1.0
...,...,...,...,...,...,...,...,...,...
763,10.0,101.0,76.0,48.0,180.0,32.9,0.171,63.0,0.0
764,2.0,122.0,70.0,27.0,0.0,36.8,0.340,27.0,0.0
765,5.0,121.0,72.0,23.0,112.0,26.2,0.245,30.0,0.0
766,1.0,126.0,60.0,0.0,0.0,30.1,0.349,47.0,1.0


In [7]:
# create model
model = KerasRegressor(model=create_model, epochs=5, batch_size=10, verbose=1)

In [8]:
model

KerasRegressor(
	model=<function create_model at 0x00000117668CF0D0>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=10
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=5
)

In [9]:
# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']

In [10]:
param_grid = dict(model__optimizer=optimizer)

In [11]:
param_grid

{'model__optimizer': ['SGD',
  'RMSprop',
  'Adagrad',
  'Adadelta',
  'Adam',
  'Adamax',
  'Nadam']}

In [12]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

In [13]:
grid_result = grid.fit(X, Y)

Epoch 1/5
77/77 [==============================] - 1s 3ms/step - loss: 2.0196 - accuracy: 0.6302
Epoch 2/5
77/77 [==============================] - 0s 3ms/step - loss: 0.6698 - accuracy: 0.6380
Epoch 3/5
77/77 [==============================] - 0s 2ms/step - loss: 0.6509 - accuracy: 0.6510
Epoch 4/5
77/77 [==============================] - 0s 2ms/step - loss: 0.6463 - accuracy: 0.6354
Epoch 5/5
77/77 [==============================] - 0s 2ms/step - loss: 0.6329 - accuracy: 0.6576


In [14]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.034744 using {'model__optimizer': 'SGD'}
-0.034744 (0.031811) with: {'model__optimizer': 'SGD'}
-0.657189 (0.258659) with: {'model__optimizer': 'RMSprop'}
-0.995670 (0.630251) with: {'model__optimizer': 'Adagrad'}
-1.081336 (0.727916) with: {'model__optimizer': 'Adadelta'}
-0.239115 (0.105434) with: {'model__optimizer': 'Adam'}
-0.842524 (0.160130) with: {'model__optimizer': 'Adamax'}
-0.322291 (0.147845) with: {'model__optimizer': 'Nadam'}
